In [1]:
# Enforces PEP-8 style guideline
!pip install --upgrade pip
!pip install flake8 pycodestyle_magic

# Adds upper path to import common module
import sys
sys.path.append("../../")

Requirement already up-to-date: pip in /opt/conda/lib/python3.8/site-packages (20.2.4)


In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [ ]:
"""
Version 1: TIMEOUT
- used methods: brute-force with some optimizations

"""

# Switch-to-clock connection
SWITCH_CONNECTIVITY = [
    [0, 1, 2],
    [3, 7, 9, 11],
    [4, 10, 14, 15],
    [0, 4, 5, 6, 7],
    [6, 7, 8, 10, 12],
    [0, 2, 14, 15],
    [3, 14, 15],
    [4, 5, 7, 14, 15],
    [1, 2, 3, 4, 5],
    [3, 4, 5, 9, 13]
]

COUNT_SWITCHES = len(SWITCH_CONNECTIVITY)

INF = 1 << 32


# MInimum push count to make all clocks point 12 o'clock
def solution(clock_state):
    if clock_state[8] != clock_state[12]:
        return -1

    clock_state = [c % 12 for c in clock_state]

    def solve(switch):
        # Base condition: checked all switches
        if switch == COUNT_SWITCHES:
            return INF if any(t != 0 for t in clock_state) else 0

        ret = INF
        for count in range(4):
            ret = min(ret, count + solve(switch + 1))
            for clock in SWITCH_CONNECTIVITY[switch]:
                clock_state[clock] = (clock_state[clock] + 3) % 12

        return ret

    return solve(0)


C = int(input())
for _ in range(C):
    clock_state = list(map(int, input().split()))
    result = solution(clock_state)
    print(result)

In [3]:
"""
Version 2
- used methods: greedy?

===================================================================================================================================
switch-clock mapping  │clock-switch map
0: 0  1  2            │0 : 0 3 5
1: 3  7  9  11        │1 : 0 8
2: 4  10 14 15        │2 : 0 5 8
3: 0  4  5  6  7      │3 : 1 6 8 9
4: 6  7  8  10 12     │4 : 2 3 7 8 9
5: 0  2  14 15        │5 : 3 7 8 9
6: 3  14 15           │6 : 3 4
7: 4  5  7  14 15     │7 : 1 3 4 7
8: 1  2  3  4  5      │8 : 4
9: 3  4  5  9  13     │9 : 1 9
                      │10: 2 4
                      │11: 1
                      │12: 4
                      │13: 9
                      │14: 2 5 6 7
                      │15: 2 5 6 7

first let's see clock 8, 11, 12, 13 in clock-switch mappiing that has only one switch connected to.
which means: only 'switch 4' can handle 'clock 8'

we will do following actions:
1. find all clocks(C) that has only one connected switch(as S)
2. remove C and put (C, S) to queue
3. remove S from all clocks
4. break loop if all clocks are visited else goto 1

result is:
clock  (8, 12) → 6 → 10 → 13 → (9, 11) → 7 → (4, 5) → 1 → 0 → 3
switch    4       3     2     9       1       7      8       0    5    6

some elements can swap their positions when satisfies some conditions.
which means: discovery of C can be random access

===================================================================================================================================

"""

# Check clocks in order
CHECK_ORDER = [
    (8,  [6, 7, 8, 10, 12]),
    (6,  [0, 4, 5, 6, 7]),
    (10, [4, 10, 14, 15]),
    (13, [3, 4, 5, 9, 13]),
    (9,  [3, 7, 9, 11]),
    (7,  [4, 5, 7, 14, 15]),
    (4,  [1, 2, 3, 4, 5]),
    (1,  [0, 1, 2]),
    (0,  [0, 2, 14, 15]),
    (3,  [3, 14, 15]),
]


# MInimum push count to make all clocks point 12 o'clock
def solution(clock_state):
    if clock_state[8] != clock_state[12]:
        return -1

    ret = 0
    for clock, related_clocks in CHECK_ORDER:
        cnt = ((12 - clock_state[clock]) % 12) // 3
        ret += cnt
        for related in related_clocks:
            clock_state[related] = (clock_state[related] + 3 * cnt) % 12

    return -1 if any(t != 0 for t in clock_state) else ret


C = int(input())
for _ in range(C):
    clock_state = list(map(int, input().split()))
    result = solution(clock_state)
    print(result)

0


24:80: E501 line too long (99 > 79 characters)


In [4]:
"""
Testing

"""
import helpers as hlpr

f = hlpr.clock(solution)
test_cases = [
    (([12, 6, 6, 6, 6, 6, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12], ),
     2),
    (([12, 9, 3, 12, 6, 6, 9, 3, 12, 9, 12, 9, 12, 12, 6, 6], ),
     9),
    (([3, 6, 6, 12, 3, 3, 9, 9, 6, 6, 6, 6, 6, 12, 6, 6], ),
     11),
    (([12, 3, 6, 12, 6, 6, 6, 12, 12, 12, 12, 3, 12, 9, 6, 6], ),
     15),
    (([3, 12, 12, 12, 3, 3, 6, 9, 9, 9, 9, 12, 9, 9, 12, 12], ),
     11),
    (([9, 3, 3, 9, 6, 12, 3, 6, 3, 3, 9, 6, 3, 9, 3, 3], ),
     12),
    (([12, 9, 6, 6, 12, 6, 6, 6, 6, 12, 12, 6, 6, 6, 9, 9], ),
     16),
    (([12, 9, 3, 3, 6, 9, 3, 6, 3, 9, 12, 3, 3, 6, 6, 6], ),
     19),
    (([3, 12, 6, 12, 6, 12, 12, 6, 3, 9, 9, 6, 3, 3, 3, 3], ),
     16),
    (([12, 12, 12, 9, 6, 12, 12, 3, 9, 3, 3, 12, 9, 3, 12, 12], ),
     19),
    (([6, 12, 12, 12, 6, 9, 3, 3, 12, 3, 9, 9, 12, 6, 12, 12], ),
     14),
    (([3, 12, 12, 9, 9, 6, 6, 12, 6, 9, 9, 9, 6, 12, 3, 3], ),
     14),
]

fmt = "Case {} expects {} but got {}"
for idx, tc in enumerate(test_cases):
    case, expected = tc
    result = f(*case)
    assert result == expected, \
        fmt.format(idx, hlpr.truncate(expected, 50), hlpr.truncate(result, 50))

hlpr.debug_msg("All tests done.")

[0.00001740] solution([12, 6, 6, 6, 6, 6, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]) → 2
[0.00001955] solution([12, 9, 3, 12, 6, 6, 9, 3, 12, 9, 12, 9, 12, 12, 6, 6]) → 9
[0.00001383] solution([3, 6, 6, 12, 3, 3, 9, 9, 6, 6, 6, 6, 6, 12, 6, 6]) → 11
[0.00001383] solution([12, 3, 6, 12, 6, 6, 6, 12, 12, 12, 12, 3, 12, 9, 6, 6]) → 15
[0.00001359] solution([3, 12, 12, 12, 3, 3, 6, 9, 9, 9, 9, 12, 9, 9, 12, 12]) → 11
[0.00001359] solution([9, 3, 3, 9, 6, 12, 3, 6, 3, 3, 9, 6, 3, 9, 3, 3]) → 12
[0.00001383] solution([12, 9, 6, 6, 12, 6, 6, 6, 6, 12, 12, 6, 6, 6, 9, 9]) → 16
[0.00001359] solution([12, 9, 3, 3, 6, 9, 3, 6, 3, 9, 12, 3, 3, 6, 6, 6]) → 19
[0.00001645] solution([3, 12, 6, 12, 6, 12, 12, 6, 3, 9, 9, 6, 3, 3, 3, 3]) → 16
[0.00001359] solution([12, 12, 12, 9, 6, 12, 12, 3, 9, 3, 3, 12, 9, 3, 12, 12]) → 19
[0.00001359] solution([6, 12, 12, 12, 6, 9, 3, 3, 12, 3, 9, 9, 12, 6, 12, 12]) → 14
[0.00001383] solution([3, 12, 12, 9, 9, 6, 6, 12, 6, 9, 9, 9, 6, 12, 3, 3]) → 14
All tests done.


In [5]:
!pip install matplotlib
"""
Performance analysis
- x-axis: n
- y-axis: O(f(n))

"""
import matplotlib as plt

_

''